## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/122-googleq-a-train-3fold-bert-base-unc-categoryv3/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/122-googleq-a-train-3fold-bert-base-unc-categoryv3/model_fold_1.h5', '/kaggle/input/122-googleq-a-train-3fold-bert-base-unc-categoryv3/model_fold_2.h5', '/kaggle/input/122-googleq-a-train-3fold-bert-base-unc-categoryv3/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512
N_CLASS_CAT = test['category'].nunique()

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output_seq = Dense(N_CLASS, activation="sigmoid", name="output_seq")(x)
    output_class = Dense(N_CLASS_CAT, activation="softmax", name="output_class")(pooled_output)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=[output_seq, output_class])
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test)[0][: ,:N_CLASS] / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.932751,0.674821,0.119975,0.555536,0.662290,0.606968,0.674387,0.579693,0.507852,...,0.912704,0.923667,0.579044,0.967566,0.962791,0.826529,0.166110,0.054223,0.783755,0.928684
1,46,0.886285,0.523687,0.008217,0.757152,0.773529,0.900763,0.542357,0.430740,0.064785,...,0.684753,0.947448,0.664143,0.958139,0.975366,0.880654,0.899054,0.099850,0.129082,0.883860
2,70,0.914322,0.718819,0.035117,0.806469,0.831509,0.897910,0.576657,0.520867,0.088972,...,0.868687,0.927763,0.632750,0.967762,0.970426,0.895992,0.155749,0.057174,0.814480,0.911733
3,132,0.858207,0.454314,0.007929,0.662201,0.766754,0.897975,0.546007,0.423810,0.138740,...,0.697699,0.942286,0.644609,0.968314,0.978677,0.887371,0.895447,0.158571,0.242336,0.901522
4,200,0.921184,0.546202,0.038587,0.886216,0.824055,0.853785,0.678429,0.601608,0.070214,...,0.719099,0.918167,0.660503,0.961656,0.960461,0.872111,0.166740,0.081903,0.580041,0.914271


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.894014,0.594932,0.033497,0.708212,0.790365,0.834522,0.581020,0.486002,0.217696,...,0.789103,0.934695,0.667292,0.961293,0.970152,0.870487,0.561167,0.116595,0.466043,0.912124
std,2812.670060,0.038953,0.122779,0.055064,0.096320,0.102855,0.131635,0.049883,0.077603,0.180433,...,0.076835,0.018443,0.042790,0.009997,0.010247,0.031186,0.315308,0.053046,0.259671,0.015773
min,39.000000,0.788602,0.369203,0.003178,0.310239,0.326927,0.178819,0.470438,0.348992,0.006315,...,0.600528,0.864135,0.518186,0.924831,0.923390,0.758268,0.008122,0.009112,0.043601,0.853898
25%,2572.000000,0.863942,0.487109,0.008428,0.653809,0.742213,0.807153,0.541287,0.429049,0.082563,...,0.727574,0.924147,0.639121,0.955074,0.964235,0.852317,0.238877,0.070021,0.234605,0.902705
50%,5093.000000,0.895274,0.584812,0.013179,0.699936,0.796542,0.881889,0.572622,0.465195,0.151225,...,0.792021,0.936972,0.665348,0.962585,0.971824,0.873528,0.666822,0.119354,0.420834,0.912986
75%,7482.000000,0.924655,0.695911,0.029515,0.765949,0.860567,0.913090,0.616657,0.521201,0.303242,...,0.855222,0.948142,0.694017,0.968386,0.978211,0.892670,0.846794,0.154087,0.670410,0.923420
max,9640.000000,0.973835,0.860832,0.441970,0.957097,0.971520,0.982222,0.706289,0.749186,0.829211,...,0.938067,0.972939,0.800639,0.980715,0.988292,0.949450,0.951799,0.274680,0.981783,0.952058
